In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import implicit

In [87]:
rating_file_path = "./data/ml-1m/ratings.dat"
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
data = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(data)

movie_file_path = "./data/ml-1m/movies.dat"
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')

In [88]:
data.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [89]:
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [90]:
data = data[data['ratings'] >= 3]
filtered_data_size = len(data)

data.rename(columns={'ratings':'counts'}, inplace=True)

In [91]:
data.head()

,user_id,movie_id,counts,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [111]:
print(movies["title"].nunique())
print(data["user_id"].nunique())
pm = data.groupby("movie_id")["counts"].sum().sort_values()[-30:].index
pp = data.groupby("movie_id")["counts"].sum().sort_values()[-30:].values

movie_id = [movies[movies["movie_id"] == i]["title"].values for i in pm]
print(movie_id)

3883
6040
[array(['Alien (1979)'], dtype=object), array(['Ghostbusters (1984)'], dtype=object), array(['Toy Story (1995)'], dtype=object), array(['Terminator, The (1984)'], dtype=object), array(['Forrest Gump (1994)'], dtype=object), array(['E.T. the Extra-Terrestrial (1982)'], dtype=object), array(['Groundhog Day (1993)'], dtype=object), array(['Being John Malkovich (1999)'], dtype=object), array(['Pulp Fiction (1994)'], dtype=object), array(['Men in Black (1997)'], dtype=object), array(['Shakespeare in Love (1998)'], dtype=object), array(['L.A. Confidential (1997)'], dtype=object), array(['Jurassic Park (1993)'], dtype=object), array(['Princess Bride, The (1987)'], dtype=object), array(['Godfather, The (1972)'], dtype=object), array(['Back to the Future (1985)'], dtype=object), array(['Shawshank Redemption, The (1994)'], dtype=object), array(['Braveheart (1995)'], dtype=object), array(["Schindler's List (1993)"], dtype=object), array(['Fargo (1996)'], dtype=object), array(['Terminato

In [93]:
my_id = 6041
my_movies_name = ["Matrix, The (1999)", "Terminator 2: Judgment Day (1991)", "Silence of the Lambs, The (1991)", "Godfather, The (1972)", "Men in Black (1997)"]
my_movies_id = [2502, 585, 589, 857, 1539]
my_rating = [5, 5, 5, 5, 5]

df = pd.DataFrame({"user_id":my_id, "movie_id":my_movies_id, "counts":my_rating, "timestamp":""})

data = data.append(df, ignore_index=True)
data.tail(10)


,user_id,movie_id,counts,timestamp
836473,6040,1090,3,956715518
836474,6040,1094,5,956704887
836475,6040,562,5,956704746
836476,6040,1096,4,956715648
836477,6040,1097,4,956715569
836478,6041,2502,5,
836479,6041,585,5,
836480,6041,589,5,
836481,6041,857,5,
836482,6041,1539,5,


In [104]:
data = data[["user_id", "movie_id", "counts"]]

csr_data = scipy.sparse.csr_matrix((data.counts, (data.user_id, data.movie_id)), shape=(max(data["user_id"])+1, max(data["movie_id"])+1))
csr_data

<6042x3953 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [105]:
import os

os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [107]:
als_model = implicit.als.AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=20, dtype=np.float32)
csr_data_transpose = csr_data.T
als_model.fit(csr_data_transpose)

  0%|          | 0/20 [00:00<?, ?it/s]

In [135]:
movie_to_index = {}
index_to_movie = {}

for i in range(len(movies)):
    movie_to_index[movies["movie_id"][i]] = movies["title"][i]
    index_to_movie[movies["title"][i]] = movies["movie_id"][i]

In [140]:
my_vector = als_model.user_factors[my_id]
matrix_vector = als_model.item_factors[index_to_movie["Matrix, The (1999)"]]
jumanji_vector = als_model.item_factors[index_to_movie["Jumanji (1995)"]]

print("매트릭스 선호도 :", np.dot(my_vector, matrix_vector))
print("주만지 선호도 :", np.dot(my_vector, jumanji_vector))

매트릭스 선호도 : 0.30065006
주만지 선호도 : -0.007658587


In [176]:
similar_movies = als_model.similar_items(index_to_movie["Matrix, The (1999)"])
print("매트릭스와 비슷한 영화")
for i in range(1, len(similar_movies)):
    print(movie_to_index[similar_movies[i][0]])

매트릭스와 비슷한 영화
Terminator 2: Judgment Day (1991)
Total Recall (1990)
Fugitive, The (1993)
Terminator, The (1984)
Fifth Element, The (1997)
Face/Off (1997)
Jurassic Park (1993)
Men in Black (1997)
Twelve Monkeys (1995)


In [167]:
similar_movies = als_model.similar_items(index_to_movie["Silence of the Lambs, The (1991)"])
print("양들의 침묵과 비슷한 영화")
for i in range(1, len(similar_movies)):
    print(movie_to_index[similar_movies[i][0]])

양들의 침묵과 비슷한 영화
Shawshank Redemption, The (1994)
Fargo (1996)
Pulp Fiction (1994)
Schindler's List (1993)
Usual Suspects, The (1995)
GoodFellas (1990)
Sixth Sense, The (1999)
Good Will Hunting (1997)
Sling Blade (1996)


In [170]:
recommand_movies = als_model.recommend(my_id, csr_data, N=20, filter_already_liked_items=True)
print("내가 좋아할만한 영화")
for i in range(1, len(recommand_movies)):
    print(movie_to_index[recommand_movies[i][0]])

내가 좋아할만한 영화
One Flew Over the Cuckoo's Nest (1975)
Jurassic Park (1993)
Fugitive, The (1993)
Braveheart (1995)
Total Recall (1990)
Terminator, The (1984)
Men in Black (1997)
South Park: Bigger, Longer and Uncut (1999)
Rushmore (1998)
Hunt for Red October, The (1990)
Kingpin (1996)
X-Men (2000)
Speed (1994)
Thomas Crown Affair, The (1999)
American Pie (1999)
Austin Powers: The Spy Who Shagged Me (1999)
Fight Club (1999)
Face/Off (1997)
Nightmare Before Christmas, The (1993)


In [175]:
fight_club_explain = als_model.explain(my_id, csr_data, itemid=index_to_movie["Fight Club (1999)"])
[(movie_to_index[i[0]], i[1]) for i in fight_club_explain[1]]

[('Office Space (1999)', 0.11803721528699598),
 ('Twin Town (1997)', 0.007003374381146878),
 ('Brady Bunch Movie, The (1995)', 0.0015861963625242329),
 ('Crows and Sparrows (1949)', 0.0008307394714064547),
 ('Terminator 2: Judgment Day (1991)', -0.0015074225567798935)]

## 결론
### implicit 패키지의 AlternatingLeastSquares 모델을 사용해서 MF모델을 만들어 봤습니다. 
### 사실  AlternatingLeastSquares 모델이 어떻게 돌아가는 지 잘 모르겠습니다. 어디서 weight가 업데이트지 그라디언트는 어디서 흐르는지
### 특이값분해에서부터 후에 나온 트랜스포머까지 전반적으로 정보이론에서 부르는 데이터의 잠재요소 혹은 특징을 찾는데 집중한다는 생각이 많이 듭니다. 퍼셉트론에서 부터 시작한 인공지능이 이제는 인공지능이라고 부르기 힘들 정도로 너무 멀리오지 않았나 하는 생각???
### 어쨋든 오늘 노드로 돌아와서
### als모델의 항목간의 유사도 측정을 보면 비교적 괜찮은 유사성을 띄는 것을 알 수 있었습니다. 다만 장르, 흥행성적, 연도등의 데이터는 제외하고 별점으로 데이터를 평가했기 때문에 '추천'이라고 까지 보기는 힘든 결과라고 생각합니다. 애초에 제가 알만한 유명 영화를 넣었기 때문에 비슷한 수준의 유명 영화가 나오는 당연한 결과겠죠. 데이터에 아는 영화가 몇개 없어서ㅠㅠㅠㅠ 정성평가도 어렵네요
### recommand의 결과로 추천된 항목을 살펴보면 제가 인풋으로 넣을 영화가 블록버스터 특성의 영화 위주로 넣어줬기 때문에 추천영화들도 유사한 영화들이 리턴되었습니다. 한번쯤 들어본 영화들이니까 선호도까지는 반영이 안되더라도 어느정도 유의미한 데이터가 나온것 같습니다.
### 마지막으로 추천된 영화중 파이트클럽의 기여도를 뽑아보았습니다. 오피스 스페이스란 영화가 가장 높은 기여도를 준 것으로 나타났습니다. 이것도 처음들어 보는 영화라 분석이 참 어렵네요.. 브래드피트랑 제니퍼애니스톤이랑 사귈때 찍은 영화라서 비슷한 사람들이 비슷한 평점줬나 싶기도 하고 그거말고는 딱히 공통점이 안보여서 뭐 그렇다고 하니깐 그런걸로